In [1]:
include("../src/modules/TUM69.jl")
include("../src/modules/Preprocess.jl")
include("../src/modules/Network.jl")
include("../src/modules/Utils.jl")

## Third party packages
using PyPlot: plot, specgram, subplot, xlabel, ylabel, title, legend
using Images
using ImageView: imshow
using Knet: minibatch

## Handwritten modules
using .TUM69: load_accel_data, load_image_data   # Data reading
using .Preprocess: process_accel_signal, process_image # Preprocessing on the data
using .Network: Conv, Dense, VisualNet, train!, accuracy # Construction of custom network
using .Utils: plot_spectrogram, extract_PCA

In [2]:
DATA_PATH = "./../data/new" 

"./../data/new"

In [3]:
X_accel_train, y_accel_train,
X_accel_test, y_accel_test, 
material_dict = load_accel_data(DATA_PATH; mode = "normal"); 

Train accel data is being loaded!
Test accel data is being loaded!


In [4]:
println("X_accel_train = ", summary(X_accel_train))
println("y_accel_train = ", summary(y_accel_train))
println("X_accel_test  = ", summary(X_accel_test))
println("y_accel_test  = ", summary(y_accel_test))
println("material_dict = ", summary(material_dict))

X_accel_train = 690-element Array{Array{Float32,1},1}
y_accel_train = 690-element Array{Int8,1}
X_accel_test  = 690-element Array{Array{Float32,1},1}
y_accel_test  = 690-element Array{Int8,1}
material_dict = Dict{String,Int8} with 69 entries


In [5]:
@time X_accel_train_modified, y_accel_train_modified = process_accel_signal(X_accel_train, y_accel_train);
@time X_accel_test_modified, y_accel_test_modified = process_accel_signal(X_accel_test, y_accel_test);

 30.480571 seconds (6.88 M allocations: 6.301 GiB, 3.08% gc time)
 21.575204 seconds (310.56 k allocations: 3.917 GiB, 3.66% gc time)


In [6]:
println("X_accel_train = ", summary(X_accel_train_modified))
println("y_accel_train = ", summary(y_accel_train_modified))
println("X_accel_test  = ", summary(X_accel_test_modified))
println("y_accel_test  = ", summary(y_accel_test_modified))
println("material_dict = ", summary(material_dict))

X_accel_train = 50×300×1×4494 Array{Float32,4}
y_accel_train = 4494-element Array{Int8,1}
X_accel_test  = 50×300×1×2325 Array{Float32,4}
y_accel_test  = 2325-element Array{Int8,1}
material_dict = Dict{String,Int8} with 69 entries
